### Import packages

In [44]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [45]:
clstr = Cluster()
session = clstr.connect()

In [46]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

bd23
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


In [47]:
session.execute("CREATE KEYSPACE IF NOT EXISTS bd23 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

In [48]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

bd23
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


In [49]:
session.execute("""
CREATE TABLE IF NOT EXISTS bd23.inventory_info ( 
    SKU INT PRIMARY KEY,
    warehouse_num INT,
    description TEXT, 
    name TEXT,
);
""")

In [50]:
df = pd.read_csv('Assignmentcsv.csv')
df

,SKU,warehouse_num,description,name
0,7777,123,'Allergy relief','cetrzn'
1,8672,123,'Cough and cold','Nyquill'
2,5634,456,'Headache','Dayquill'
3,6726,349,'Body Pains','Tylenol'
4,7898,789,'Cough Syrup','RObitsu'


In [51]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO bd23.inventory_info (SKU,warehouse_num, description, name)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )
    session.execute(f"""
        INSERT INTO bd23.inventory_info (SKU,warehouse_num, description, name)    
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )


        INSERT INTO bd23.inventory_info (SKU,warehouse_num, description, name)     
        VALUES (7777, 123, 'Allergy relief', 'cetrzn');
        

        INSERT INTO bd23.inventory_info (SKU,warehouse_num, description, name)     
        VALUES (8672, 123, 'Cough and cold', 'Nyquill');
        

        INSERT INTO bd23.inventory_info (SKU,warehouse_num, description, name)     
        VALUES (5634, 456, 'Headache', 'Dayquill');
        

        INSERT INTO bd23.inventory_info (SKU,warehouse_num, description, name)     
        VALUES (6726, 349, 'Body Pains', 'Tylenol');
        

        INSERT INTO bd23.inventory_info (SKU,warehouse_num, description, name)     
        VALUES (7898, 789, 'Cough Syrup', 'RObitsu');
        


In [55]:
rows = session.execute("select (SKU,warehouse_num, description, name) from bd23.inventory_info")
for row in rows:
    print(f"SKU={row[0][0]}, warehouse_num{row[0][1]}, description={row[0][2]}, name={row[0][3]}")

SKU=7777, warehouse_num123, description=Allergy relief, name=cetrzn
SKU=8672, warehouse_num123, description=Cough and cold, name=Nyquill
SKU=7898, warehouse_num789, description=Cough Syrup, name=RObitsu
SKU=6726, warehouse_num349, description=Body Pains, name=Tylenol
SKU=5634, warehouse_num456, description=Headache, name=Dayquill


Search for all products located at warehouse_num =123

In [57]:
rows = session.execute("select (SKU, warehouse_num, name) from bd23.inventory_info where warehouse_num = 123 ALLOW FILTERING")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}")

7777, 123, cetrzn
8672, 123, Nyquill
